In [22]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [23]:
rent_obj = []
all_property = []

for page in range(9,21):
    link = 'https://www.apartments.com/condos/ca/' + str(page) + '/'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
#     page 1-9
#     results = soup.find_all('article', class_='premiumPlus placard')
#     page 9-28
    results = soup.find_all('article', class_='premium placard')

    # Loop through returned results and get the hyperlink of the properties of the page
    for result in results:
        find_tag = result.find('a')['href']
        new_response = requests.get(find_tag, headers=headers)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')
        new_results = new_soup.find('div', class_="mainWrapper")
        
        full_address = result.find('div', class_ = "location").text
        city = new_results.find('div', class_ = "propertyAddress").select('span')[0].text
        state = new_results.find('div', class_ = "propertyAddress").select('span')[1].text
        zip_code = new_results.find('div', class_ = "propertyAddress").select('span')[2].text
        street = new_results.find('h1', class_ = "propertyName").text.replace('\r\n                    ', "").strip()
        bedroom_container = new_results.find('div', class_="rentRollupContainer")
        house_details = new_results.find('table', class_="availabilityTable")
        bath_number = house_details.find('td', class_="baths").find('span', class_="shortText").text.replace('\r\n                    ', "").strip()
        house_square_feet = house_details.find('td', class_="sqft").text
        square_feet = house_square_feet.split(" ")[0];
        try:
            lease_length = house_details.find('td', class_="leaseLength").text
        except:
            lease_length = 'null'
        availability_check = house_details.find('td', class_="available").text.replace('\r\n                    ', "").strip()
        all_the_rents = bedroom_container.findAll('span', class_="rentRollup")

        bedroom_info = []
        for rent_block in all_the_rents:
            bedroom_type = rent_block.find('span', class_="shortText").text
            try:
                price = re.search('\$.* ', rent_block.text)
                price = price.group(0)[:-3]

            except:
                if "Call for Rent" in rent_block.text:
                    price = ""
                else:
                    price = re.search('\$.*', rent_block.text).group()[:-1]
            bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
        rent_obj = {
                "City_Rent": city,
                "Type": "Condos",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                "Full_Address": full_address,
                "Rent_link": find_tag,
                "Bath_Info": bath_number,
                "Square_Feet_Data": house_square_feet,
                "Square_Feet": square_feet,
                "Lease_length": lease_length,
                "Availability_check": availability_check
            }
        for room_type in bedroom_info:
            rent_obj[room_type["bedroom_type"]] = room_type["price"];
        
        all_property.append(rent_obj)
        time.sleep(1)
print(all_property)

[{'City_Rent': 'Davis', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '95616', 'Bedroom_Info': [{'bedroom_type': '1 Bed', 'price': '$95'}], 'Full_Address': 'Davis, CA 95616', 'Rent_link': 'https://www.apartments.com/819-h-st-davis-ca/kqwkkwc/', 'Bath_Info': '1 BA', 'Square_Feet_Data': '1,500 Sq Ft', 'Square_Feet': '1,500', 'Lease_length': '11 Month Lease', 'Availability_check': 'Available Now', '1 Bed': '$95'}, {'City_Rent': 'Los Angeles', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '91423', 'Bedroom_Info': [{'bedroom_type': 'Studio', 'price': '$2,00'}], 'Full_Address': 'Los Angeles, CA 91423', 'Rent_link': 'https://www.apartments.com/14431-dunbar-pl-los-angeles-ca/ql0flyz/', 'Bath_Info': '1 BA', 'Square_Feet_Data': '405 Sq Ft', 'Square_Feet': '405', 'Lease_length': '12 Month Lease', 'Availability_check': 'Available Now', 'Studio': '$2,00'}, {'City_Rent': 'Pomona', 'Type': 'Condos', 'State_Rent': 'CA', 'Zip_code': '91766', 'Bedroom_Info': [{'bedroom_type': 'Studio', 'price': 

In [24]:
rent_data = pd.DataFrame(all_property)
rent_data

,City_Rent,Type,State_Rent,Zip_code,Bedroom_Info,Full_Address,Rent_link,Bath_Info,Square_Feet_Data,Square_Feet,Lease_length,Availability_check,1 Bed,Studio,2 Beds,3 Beds,5 Beds,4 Beds
0,Davis,Condos,CA,95616,"[{'bedroom_type': '1 Bed', 'price': '$95'}]","Davis, CA 95616",https://www.apartments.com/819-h-st-davis-ca/k...,1 BA,"1,500 Sq Ft","1,500",11 Month Lease,Available Now,$95,NaN,NaN,NaN,NaN,NaN
1,Los Angeles,Condos,CA,91423,"[{'bedroom_type': 'Studio', 'price': '$2,00'}]","Los Angeles, CA 91423",https://www.apartments.com/14431-dunbar-pl-los...,1 BA,405 Sq Ft,405,12 Month Lease,Available Now,NaN,"$2,00",NaN,NaN,NaN,NaN
2,Pomona,Condos,CA,91766,"[{'bedroom_type': 'Studio', 'price': '$2,30'}]","Pomona, CA 91766",https://www.apartments.com/855-w-2nd-st-pomona...,2 BAs,"1,000 Sq Ft","1,000",12 Month Lease,Available Now,NaN,"$2,30",NaN,NaN,NaN,NaN
3,Los Angeles,Condos,CA,90023,"[{'bedroom_type': '1 Bed', 'price': '$1,50'}]","Los Angeles, CA 90023",https://www.apartments.com/1356-s-sunol-dr-los...,1 BA,400 Sq Ft,400,12 Month Lease,Available Now,"$1,50",NaN,NaN,NaN,NaN,NaN
4,San Diego,Condos,CA,92130,"[{'bedroom_type': '2 Beds', 'price': '$2,42'}]","San Diego, CA 92130",https://www.apartments.com/3617-bernwood-pl-sa...,2 BAs,950 Sq Ft,950,12 Month Lease,Available Now,NaN,NaN,"$2,42",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,San Diego,Condos,CA,92111,"[{'bedroom_type': '2 Beds', 'price': '$1,75'}]","San Diego, CA 92111",https://www.apartments.com/4122-mt-alifan-dr-s...,1 BA,900 Sq Ft,900,12 Month Lease,Available Now,NaN,NaN,"$1,75",NaN,NaN,NaN
194,Richmond,Condos,CA,94804,"[{'bedroom_type': '2 Beds', 'price': '$2,85'}]","Richmond, CA 94804",https://www.apartments.com/565-hayes-st-richmo...,1 BA,950 Sq Ft,950,12 Month Lease,Available Now,NaN,NaN,"$2,85",NaN,NaN,NaN
195,Lake Forest,Condos,CA,92630,"[{'bedroom_type': '2 Beds', 'price': '$2,25'}]","Lake Forest, CA 92630",https://www.apartments.com/22874-bonita-ln-lak...,1½ BAs,"1,020 Sq Ft","1,020",1 Month Lease,Available Now,NaN,NaN,"$2,25",NaN,NaN,NaN
196,Bakersfield,Condos,CA,93308,"[{'bedroom_type': '2 Beds', 'price': '$1,00'}]","Bakersfield, CA 93308",https://www.apartments.com/6524-mohawk-st-bake...,1 BA,700 Sq Ft,700,6 Month Lease,Available Now,NaN,NaN,"$1,00",NaN,NaN,NaN


In [25]:
# Export file as a CSV, without the Pandas index, but with the header
rent_data.to_csv("Output/Apartment.com/Q3-2020_Apartment.com_Condos_Data_page9-20.csv", index=False, header=True)